# 準備
## ライブラリのインストール
Colaboratory環境で実行する場合は`japanaize_matplotlib`をインストールする必要があります。

SageMaker環境で以下を実行する必要はありませんが、実行しても害はありません。

In [ ]:
%pip install japanize_matplotlib

## ライブラリのインポートとデータの読み込み

この資料で使用するライブラリを一括してインポートし、必要なデータを読み込みます。

SageMakerやColaboratoryの接続がタイムアウトした時は、
以下のセルを再実行して下さい。

In [ ]:
# ライブラリーのインポート
import sklearn
import numpy as np
from sklearn import manifold
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import japanize_matplotlib
import math
import random
import pandas as pd

# データセットの読み出し
akutagawa_name = np.load('data/akutagawa_name.npy', allow_pickle=True)
kikuchi_name = np.load('data/kikuchi_name.npy', allow_pickle=True)
hgram = np.load('data/hgram.npy', allow_pickle=True)
words = np.load('data/words.npy', allow_pickle=True)
print('芥川龍之介の作品は以下の20編')
print(' '.join(akutagawa_name))
print('\n菊池寛の作品は以下の20編')
print(' '.join(kikuchi_name))
print('\n40編の作品に現れる語の総数は{}語'.format(len(words)))

## 2.12 データの加工と類似度関数の設計(第3ラウンド)

ここでは、類似度関数として、ユークリッド距離に替えて、マンハッタン距離を利用することで正解率の改善を目指します。



### $L^1$ノルム
類似度関数に$L^1$距離を使用するモチベーションについて述べます。はじめに、ベクトル$v,w$の$L^1$距離は以下で計算されます。

$$
d_{L1}((v_1, \dots, v_{16301}), (w_1, \dots, w_{16301})) = 
\Vert (v_1, \dots, v_{16301}) - (w_1, \dots, w_{16301})\Vert_1 =
\sum_{i=1}^{16301} \vert v_i - w_i \vert 
$$

$L^1$ノルムと$L^2$ノルムの違いを以下のようにまとめることができます。

>**$L^1$ノルムと$L^2$ノルムの比較**
>
>$L^1$ノルムと$L^2$ノルムを比較すると、相対的に次の傾向が見られます。
>
>*  $L^1$ノルムは、ベクトルが0に近い成分を多く持つ時、たとえ、
その他の成分の絶対値が比較的大きい場合でも小さく評価する
>*  $L^2$ノルムは、成分の絶対値が平均的に小さいベクトルを小さく評価する

詳しくは「L2距離とL1距離.ipynb」を参照して下さい。

### 文体に関する適用した時の$L^2$ノルムとの違い

作品に現れる語彙には次の2種類があると考えられます。

1. **作品に固有の語彙**.
同一の著者による作品の間でも、作品毎に作品中での出現確率が大きく異なることが想定されます。
1. **作品によらず普遍的に現れる語彙**.
著者の文体は、この語彙の単語の作品中での出現確率によって特徴付けられる、
即ち、この語彙の単語の分布は著者によって一定の傾向を示すことが想定される。

言い換えると、$\vec v_1$と$\vec v_2$を、
同一の著者の2編の作品に対する単語の出力確率を表すベクトル（確率分布）とします。
この時、$\vec v_1$と$\vec v_2$の差$\vec v_1 - \vec v_2$の成分について考察しましょう。

1. **作品に固有の語彙**の単語では、$\vec v_1 - \vec v_2$の成分は0から遠い値になります。
1. **作品によらず普遍的に現れる語彙**の単語では、$\vec v_1 - \vec v_2$の成分は0に近い値になります。

つまり、同じ著者による作品の確率分布ベクトル$\vec v_1 - \vec v_2$の長さ（ノルム）は、
$L^1$ノルムを用いた方が$L^2$ノルムを用いた場合よりも小さく評価され、
**同一の著者の作品をより近く、異なる著者の作品をより遠く**評価されること分かります。



### $L^1$正規化と確率分布
今回は類似度関数に$L^1$距離(マンハッタン距離)を使用することにしたため、正規化でも$L^1$正規化を行うことにします。

$$
  \frac{\vec v}{\Vert\vec v\Vert_1} =
  \left(\frac{v_1}{\vert v_1\vert + \dots + \vert v_{16301}\vert}, \dots,
    \frac{v_{16301}}{\vert v_1\vert + \dots + \vert v_{16301}\vert}\right)
$$

$L^1$正規化で得られる値は語の出現確率になります。

>- 全ての成分は0以上で、総和は1になるからです。
>- 以下のコードでは`hgrm`の要素を$L_1$正規化して、リスト`prob`を生成します。\
`hgrm[i]`の頻度情報を$L^1$正規化した値は`prob[i]`に保存します。



In [ ]:
def dl1(dictx, dicty): # L1距離
    sq = 0
    for key in set(dictx.keys()) | set(dicty.keys()):
        x, y = 0, 0
        if key in dictx: 
            x = dictx[key]
        if key in dicty:
            y = dicty[key]
        sq += abs(x - y)
    return sq

def distribution(dct): # ヒストグラムを確率分布に変換
    s = sum(dct.values())
    return dict([(k, v/s) for k, v in dct.items()]) 

prob = list(map(lambda dct: distribution(dct), hgram)) # 確率分布データ
print(prob)

「蜘蛛の糸」を例に、$L^2$正規化後の分布と、$L^1$正規化後の分布を比較してみます。

In [ ]:
def l2normalize(dct): # L2正規化
    s = np.sqrt(sum([v**2 for v in dct.values()]))
    return dict([(k, v/s) for k, v in dct.items()]) 

hgrm_nrm = [l2normalize(dct) for dct in hgram] # hgramは前に作成したリスト。各作品の単語の種類とその単語の出現回数を持つ辞書型オブジェクトを作品の数だけ持つ。

def countl2(n, w): # n番目の作品にwという単語が何回出現しているかを返す関数
    if w in hgrm_nrm[n]:
        return hgrm_nrm[n][w]
    else:
        return 0

def countl1(n, w): # n番目の作品において、無作為に名詞・動詞・形容詞・副詞の中から一つ単語を抽出したときwという単語である確率を返す関数
    if w in prob[n]:
        return prob[n][w]
    else:
        return 0

# L1正規化とL2正規化の分布の比較
_, axes = plt.subplots(1, 2, figsize=[15,5])

y = [countl2(19, w) for w in words] # 19は蜘蛛の糸に対応
axes[0].plot(range(len(words)), y)
axes[0].set_title("蜘蛛の糸($L_2$正規化)")
axes[0].set_ylim([0, 0.3])
axes[0].xaxis.set_visible(False)

y = [countl1(19, w) for w in words]
axes[1].plot(range(len(words)), y)
axes[1].set_title("蜘蛛の糸($L_1$正規化)")
axes[1].set_ylim([0, 0.3])
axes[1].xaxis.set_visible(False)

plt.show()


前回のサイクルと同様に次元圧縮後、3次元表示して、感触をつかみます。

In [ ]:
 # L1距離を用いた距離行列の作成とMDSによる圧縮&可視化
temp = [] 
for dictx in prob:
    temp.append(list(map(lambda dicty: dl1(dictx, dicty), prob)))
dl1_matrix = np.array(temp) # 距離の行列

dl1_matrix

mds = manifold.MDS(n_components=3, dissimilarity="precomputed", random_state=6)
pos = mds.fit_transform(dl1_matrix)

l1 = len(akutagawa_name)
l2 = len(kikuchi_name)

fig = go.Figure(
    layout=go.Layout(
    title="L1正規化後、マンハッタン距離に基づくデータの分布(MDSで次元圧縮)",
    showlegend=True,
    legend=dict(x=0.7, y=0.99, xanchor='left', yanchor='top', font=dict(size=16))
    )
)

fig.add_trace(go.Scatter3d(
        x = pos[0: l1,0], y = pos[0: l1,1], z = pos[0: l1,2],
        mode = "markers",
        marker = dict(symbol="cross", color="black", size=5, 
                      line=dict(width=0), opacity=1 ),
        name = "芥川"
        )
)

fig.add_trace(go.Scatter3d(
        x = pos[l1: l1+l2, 0], y = pos[l1: l1+l2, 1], z = pos[l1: l1+l2, 2],
        mode = "markers",
        marker = dict(symbol="circle", color="black", size=5, 
                      line=dict(width=0), opacity=1),
        name = "菊池"
        )
)

分布が明確に分離し、分布の間の間隙もはっきりしているように見えます。
つまり、k-NNによる分類に適した分布の様相を示しており、期待ができます。

## 2.13 学習アルゴリズムの選択(第3ラウンド)
2.12節でのMDSの結果を見ると、k-NNアルゴリズムでの正解率の向上が期待できます。

## 2.14 ハイパーパラメータの最適化とモデルの評価(第3ラウンド)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8, 6))

labels = [0]*l1 + [1]*l2

parameters = [{"metric": ["precomputed"], "n_neighbors": list(range(1,11))}]
clf = GridSearchCV(KNeighborsClassifier(), parameters, cv=5)
clf.fit(dl1_matrix, labels)

x = []
y = []

params = clf.cv_results_["params"]
mean_test_score = clf.cv_results_["mean_test_score"]
std_test_score = clf.cv_results_["std_test_score"]
for p, m, s in zip(params, mean_test_score, std_test_score):
    print(f"{m:.3f} (+/- {s/2:.3f}) for {p}")
    x.append(p['n_neighbors'])
    y.append(m)



plt.ylim(0, 1)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel("$k$", fontsize=16)
plt.ylabel("正解率", fontsize=16)
plt.bar(x, y)
plt.grid()
plt.show()


$k$の最適値と、正解率の最大値は以下の通りです。

In [ ]:
print(f"kの最適値:\t{clf.best_estimator_.n_neighbors}")
scores = model_selection.cross_val_score(clf.best_estimator_, X = dl1_matrix, y = labels, cv = 5)
print(f"k={clf.best_estimator_.n_neighbors}での正解率:\t{np.average(scores)}")

良好な正解率を得ることができました。

## 2.15 まとめ

芥川龍之介の作品と菊池寛の作品をテキストのみに基づいて判別する簡単な「人工知能」を作成しました。

- 「自立語である名詞・動詞・形容詞・副詞の分布が著者の文体を特徴付ける」という仮説に基づく。
- データをベクトルと考え、ノルム（（$L^2$ノルム・$L^1$ノルム）による類似度関数を試した。
- データを正規化（$L^2$正規化・$L^1$正規化）することで、作品の長さが与える悪影響を除去することを狙った。
- 結果、$L^1$ノルムによる類似度関数、データの$L^1$正規化、k-NNアルゴリズムの利用により、予測の正解率を満足できるレベルに高めることができることを発見した。

高い正解率に到達するまでに、問題を考察しつつ、試行錯誤による探索を行った点が重要です。